In [13]:
import torch
from pretrain_trfm import TrfmSeq2seq
from pretrain_rnn import RNNSeq2Seq
#from bert import BERT
from build_vocab import WordVocab
from utils import split

pad_index = 0
unk_index = 1
eos_index = 2
sos_index = 3
mask_index = 4

vocab = WordVocab.load_vocab('/home/yogesh/data/dnndr2/processedData/vocab_24_100.pkl')

trfm = TrfmSeq2seq(len(vocab), 256, len(vocab), 3)
#model = nn.DataParallel(model)
trfm.load_state_dict(torch.load('/home/yogesh/projects/202107_dnndr2/.save/trfm_new_1_0.pt'), strict=False)
trfm.eval()
print('Total parameters:', sum(p.numel() for p in trfm.parameters()))

Total parameters: 3189805


In [14]:
import pandas as pd
df = pd.read_csv('/home/yogesh/data/dnndr2/rawData/drug_smiles.csv')

In [21]:
def get_array(smiles):
    x_id, x_seg = [], []
    for sm in smiles:
        a,b = get_inputs(sm)
        x_id.append(a)
        x_seg.append(b)
    return torch.tensor(x_id), torch.tensor(x_seg)

def get_inputs(sm):
    seq_len = 220
    sm = sm.split()
    if len(sm)>218:
        print('SMILES is too long ({:d})'.format(len(sm)))
        sm = sm[:109]+sm[-109:]
    ids = [vocab.stoi.get(token, unk_index) for token in sm]
    ids = [sos_index] + ids + [eos_index]
    seg = [1]*len(ids)
    padding = [pad_index]*(seq_len - len(ids))
    ids.extend(padding), seg.extend(padding)
    return ids, seg

x_split = [split(sm) for sm in df['canonical_smiles'].values]
xid, xseg = get_array(x_split)
X = trfm.encode(torch.t(xid))
print(X.shape)

SMILES is too long (226)
SMILES is too long (248)
SMILES is too long (405)
SMILES is too long (219)
SMILES is too long (227)
SMILES is too long (375)
SMILES is too long (227)
SMILES is too long (221)
SMILES is too long (236)
SMILES is too long (232)
SMILES is too long (226)
SMILES is too long (247)
SMILES is too long (342)
SMILES is too long (225)
SMILES is too long (225)
SMILES is too long (226)
SMILES is too long (229)
SMILES is too long (227)
SMILES is too long (222)
SMILES is too long (230)
SMILES is too long (228)
SMILES is too long (237)
SMILES is too long (230)
SMILES is too long (228)
SMILES is too long (220)
SMILES is too long (258)
SMILES is too long (286)
SMILES is too long (288)
SMILES is too long (229)
There are 30654 molecules. It will take a little time.
(30654, 1024)


In [22]:
X[:10]

array([[-0.9648697 , -1.1149548 ,  0.73070174, ...,  2.2982287 ,
        -1.3199449 ,  1.3015112 ],
       [-0.9388767 , -1.4205409 ,  1.0319326 , ...,  2.0778277 ,
        -1.2846612 ,  1.2787257 ],
       [-0.89688015, -1.758795  ,  1.4770942 , ...,  2.0211818 ,
        -1.2406851 ,  1.1257133 ],
       ...,
       [-1.2045133 , -0.71763146,  1.5659194 , ...,  2.4852614 ,
        -1.3761665 ,  1.7818329 ],
       [-0.84381795, -1.1360548 ,  1.6008846 , ...,  2.09459   ,
        -1.4096613 ,  1.9956287 ],
       [-0.86022496, -1.2613864 ,  1.5189482 , ...,  1.7429534 ,
        -1.3853273 ,  2.0293925 ]], dtype=float32)